In [1]:
import pandas as pd

In [2]:
table = pd.read_csv("gs://nflis_data/MCM_NFLIS_Data.csv")

In [3]:
table.head()

,YYYY,State,COUNTY,FIPS_State,FIPS_County,FIPS_Combined,SubstanceName,DrugReports,TotalDrugReportsCounty,TotalDrugReportsState
0,2010,VA,ACCOMACK,51,1,51001,Propoxyphene,1,84,41462
1,2010,OH,ADAMS,39,1,39001,Morphine,9,527,70999
2,2010,PA,ADAMS,42,1,42001,Methadone,2,334,89981
3,2010,VA,ALEXANDRIA CITY,51,510,51510,Heroin,5,427,41462
4,2010,PA,ALLEGHENY,42,3,42003,Hydromorphone,5,8500,89981


In [4]:
fips_set = set(table['FIPS_Combined'].tolist())

def get_fips_dict(start_year, end_year):
    # dict to calculate the total drug reports in each county
    fips_dict = {}
    for fip in fips_set:
        fips_dict[fip] = 0

    for fip in fips_dict:
        df_fip = table[table.FIPS_Combined == fip]
        df_fip = df_fip[df_fip.YYYY <= end_year]
        df_fip = df_fip[df_fip.YYYY >= start_year]
        s = set()
        s = list(set(df_fip['TotalDrugReportsCounty']))
        total = sum(s)
        fips_dict[fip] = total

    # conver the dict to list. (fips, num_of_reports)
    fips_dict = sorted(fips_dict.items(), key=lambda item: item[1], reverse=True)
    
    return fips_dict

drug_name = set(table['SubstanceName'].tolist())
drug_name = list(drug_name)

In [9]:
fips_dict = get_fips_dict(2010, 2015)
fips_dict_validation = get_fips_dict(2016,2017)
fips_dict[:10]

[(42101, 146506),
 (39035, 73728),
 (39061, 72747),
 (42003, 51509),
 (39113, 49787),
 (39049, 41053),
 (21111, 30405),
 (42045, 29726),
 (42017, 26408),
 (42133, 21655)]

In [10]:
fips_dict_validation[:10]

[(42101, 40808),
 (39035, 39602),
 (39061, 39450),
 (39049, 18862),
 (39113, 17376),
 (42003, 17272),
 (39085, 11864),
 (42045, 10346),
 (39151, 9385),
 (21111, 8460)]

In [86]:
print(drug_name)

['Isobutyryl fentanyl', 'p-Fluorofentanyl', 'o-Fluorofentanyl', 'Phenyl fentanyl', 'Codeine', 'Levorphanol', 'p-methoxybutyryl fentanyl', 'U-48800', 'Oxymorphone', 'Thebaine', 'U-49900', 'Opiates', 'Carfentanil', 'Pethidine', '3-Fluorofentanyl', 'U-51754', 'Dextropropoxyphene', '3,4-Methylenedioxy U-47700', 'Remifentanil', 'Hydrocodeinone', 'Fluorobutyryl fentanyl ', 'Crotonyl fentanyl', 'Fluoroisobutyryl fentanyl', 'Oxycodone', 'Tramadol', 'ANPP', 'Alphaprodine', 'Furanyl fentanyl', 'cis-3-methylfentanyl', 'Cyclopropyl/Crotonyl Fentanyl', 'Dihydromorphone', 'Hydromorphone', 'Heroin', 'Buprenorphine', 'Cyclopropyl fentanyl', 'U-47700', 'Methadone', 'Metazocine', 'Butyryl fentanyl', 'trans-3-Methylfentanyl', 'Butorphanol', 'Meperidine', 'p-Fluorobutyryl fentanyl', 'Methoxyacetyl fentanyl', 'Acetyldihydrocodeine', '4-Methylfentanyl', 'Mitragynine', 'Dihydrocodeine', 'Benzylfentanyl', 'MT-45', 'Morphine', 'Opium', 'Valeryl fentanyl', 'Methorphan', 'Acryl fentanyl', 'Pentazocine', '3-Methy

In [16]:
def get_data_from_FIPS_combined(start_year, end_year, fips, drug_num=5):
    df = table[table.FIPS_Combined == fips]
    df = df[df.YYYY <= end_year]
    df = df[df.YYYY >= start_year]
    df_dict = {}
    for name in drug_name:
        df_drug = df[df.SubstanceName == name]
        df_dict[name] = sum(df_drug['DrugReports'])
    df_dict = sorted(df_dict.items(), key=lambda item: item[1], reverse=True)
    df_list = []
    # We consider top 5 drugs by default
    for i in range(drug_num): 
        name = df_dict[i][0]
        print(name)
        df_with_substance = df[df.SubstanceName == name]
        report_num = list(df_with_substance['DrugReports'])
        while len(report_num) <  6:
            report_num.insert(0,0)
        df_list.append(report_num)
    return df_list

In [17]:
get_data_from_FIPS_combined(start_year=2010, end_year=2015,fips=39061,drug_num=5)

Heroin
Oxycodone
Fentanyl
Hydrocodone
Buprenorphine


[[1917, 2248, 3291, 3458, 3686, 4167],
 [1016, 324, 305, 269, 230, 250],
 [0, 11, 4, 5, 189, 1434],
 [347, 168, 126, 109, 104, 88],
 [106, 55, 41, 75, 78, 79]]

In [44]:
from sklearn import linear_model
import numpy as np
model = linear_model.LinearRegression()
X = [[2010], [2011], [2012], [2013], [2014], [2015]]
# X_new = [[2016], [2017]]
X_new = [[2010], [2011], [2012], [2013], [2014], [2015], [2016], [2017]]
def linear_regression(df_list):
    df_pred_list = []
    for data in df_list:
        model.fit(X,data)
        pred = model.predict(X_new)
        df_pred_list.append(list(pred))
    return df_pred_list

In [45]:
print(linear_regression(get_data_from_FIPS_combined(start_year=2010, end_year=2015,fips=39061)))

Heroin
Oxycodone
Fentanyl
Hydrocodone
Buprenorphine
[[2004.1904761905316, 2453.647619047668, 2903.104761904804, 3352.5619047619402, 3802.0190476190764, 4251.476190476213, 4700.933333333349, 5150.390476190485], [695.2857142857101, 576.7714285714319, 458.25714285712456, 339.74285714284633, 221.2285714285681, 102.71428571428987, -15.800000000017462, -134.3142857142957], [-276.52380952384556, -56.380952381005045, 163.76190476189367, 383.9047619047342, 604.0476190475747, 824.1904761904734, 1044.333333333314, 1264.4761904761544], [264.4285714285652, 221.4571428571362, 178.48571428570722, 135.51428571427823, 92.54285714284924, 49.571428571420256, 6.599999999991269, -36.37142857143772], [74.6190476190477, 73.70476190476188, 72.79047619047606, 71.87619047619046, 70.96190476190463, 70.04761904761904, 69.13333333333321, 68.21904761904761]]


In [35]:
drug_text = ["Heroin", "Oxycodone", "Buprenorphine", "Hydrocodone", "Fentanyl"]
county_text = [42101, 39035, 42003]

In [36]:
temp = table[table.FIPS_Combined == county_text[2]]
temp = temp[temp.SubstanceName == drug_text[4]]
temp

,YYYY,State,COUNTY,FIPS_State,FIPS_County,FIPS_Combined,SubstanceName,DrugReports,TotalDrugReportsCounty,TotalDrugReportsState
107,2010,PA,ALLEGHENY,42,3,42003,Fentanyl,3,8500,89981
2738,2011,PA,ALLEGHENY,42,3,42003,Fentanyl,2,8004,86793
5324,2012,PA,ALLEGHENY,42,3,42003,Fentanyl,4,8196,78577
8786,2013,PA,ALLEGHENY,42,3,42003,Fentanyl,5,8585,72096
12168,2014,PA,ALLEGHENY,42,3,42003,Fentanyl,68,9286,77318
13935,2015,PA,ALLEGHENY,42,3,42003,Fentanyl,312,8938,75351
16757,2016,PA,ALLEGHENY,42,3,42003,Fentanyl,770,9025,72376
20311,2017,PA,ALLEGHENY,42,3,42003,Fentanyl,1516,8247,68751


In [37]:
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
model = Ridge()
X_temp = [[0], [1], [2], [3], [4], [5]]
X_new_temp = [[0], [1], [2], [3], [4], [5], [6], [7]]
X = [[2010], [2011], [2012], [2013], [2014], [2015]]
X_new = [[2010], [2011], [2012], [2013], [2014], [2015], [2016], [2017]]

In [38]:
# data1 = [
#     [3910, 3508, 3659, 3519, 3297, 4985], 
#     [1513, 1715, 1479, 1146, 1060, 1135], 
#     [164, 144, 149, 143, 139, 132], 
#     [194, 147, 147, 96, 88, 55], 
#     [7, 7, 6, 8, 29, 166]
# ]

# data2 = [
#     [462, 537, 1877, 2277, 2703, 2771], 
#     [251, 223, 425, 436, 397, 402], 
#     [36, 21, 59, 54, 83, 105], 
#     [81, 59, 151, 142, 124, 125], 
#     [1, 2, 1, 2, 118, 286]
# ]

# data3 = [
#     [2592, 2298, 2739, 3279, 3774, 3674], 
#     [280, 317, 278, 249, 259, 172], 
#     [236, 263, 218, 234, 333, 315], 
#     [198, 201, 187, 154, 128, 105], 
#     [3, 2, 4, 5, 68, 312]
# ]

In [39]:
def linear_regression_ridge(df_list):
    df_pred_list = []
    for data in df_list:
        model.fit(X,data)
        pred = model.predict(X_new)
        df_pred_list.append(list(pred))
    return df_pred_list

In [40]:
print(linear_regression_ridge(get_data_from_FIPS_combined(start_year=2010, end_year=2015, fips=42101)))

Heroin
Oxycodone
Codeine
Buprenorphine
Hydrocodone
[[3502.0540540540533, 3626.432432432426, 3750.810810810799, 3875.189189189172, 3999.567567567574, 4123.945945945947, 4248.32432432432, 4372.7027027026925], [1624.306306306331, 1511.11711711713, 1397.927927927929, 1284.738738738757, 1171.549549549556, 1058.3603603603842, 945.1711711711832, 831.9819819819822], [271.2342342342454, 234.20720720721874, 197.1801801801921, 160.15315315316548, 123.12612612613884, 86.09909909911221, 49.072072072085575, 12.045045045058941], [157.39639639639609, 152.50450450450444, 147.6126126126128, 142.72072072072115, 137.8288288288295, 132.93693693693604, 128.0450450450444, 123.15315315315274], [183.53153153153107, 158.58558558558434, 133.6396396396376, 108.69369369369088, 83.74774774774414, 58.80180180179741, 33.85585585585795, 8.909909909911221]]


In [41]:
for fips, num in fips_dict:
    print(fips)
    print(table.loc[table.FIPS_Combined==fips,'COUNTY'].values[0] + ',' + table.loc[table.FIPS_Combined==fips,'State'].values[0])
    print(linear_regression(get_data_from_FIPS_combined(start_year=2010, end_year=2015, fips=fips)))
    print('\n')

42101
PHILADELPHIA,PA
Heroin
Oxycodone
Codeine
Buprenorphine
Hydrocodone
[[3502.0540540540533, 3626.432432432426, 3750.810810810799, 3875.189189189172, 3999.567567567574, 4123.945945945947, 4248.32432432432, 4372.7027027026925], [1624.306306306331, 1511.11711711713, 1397.927927927929, 1284.738738738757, 1171.549549549556, 1058.3603603603842, 945.1711711711832, 831.9819819819822], [271.2342342342454, 234.20720720721874, 197.1801801801921, 160.15315315316548, 123.12612612613884, 86.09909909911221, 49.072072072085575, 12.045045045058941], [157.39639639639609, 152.50450450450444, 147.6126126126128, 142.72072072072115, 137.8288288288295, 132.93693693693604, 128.0450450450444, 123.15315315315274], [183.53153153153107, 158.58558558558434, 133.6396396396376, 108.69369369369088, 83.74774774774414, 58.80180180179741, 33.85585585585795, 8.909909909911221]]


39035
CUYAHOGA,OH
Heroin
Oxycodone
Hydrocodone
Fentanyl
Buprenorphine
[[525.0180180179887, 1023.4774774773978, 1521.936936936807, 2020.39639

In [ ]:
for fips, num in fips_dict:
    print(fips)
    print(table.loc[table.FIPS_Combined==fips,'COUNTY'].values[0] + ',' + table.loc[table.FIPS_Combined==fips,'State'].values[0])
    print(linear_regression_ridge(get_data_from_FIPS_combined(fips)))
    print('\n')

42101
PHILADELPHIA,PA
Heroin
Oxycodone
Codeine
Buprenorphine
Hydrocodone
[[3502.0540540540533, 3626.432432432426, 3750.810810810799, 3875.189189189172, 3999.567567567574, 4123.945945945947, 4248.32432432432, 4372.7027027026925], [1624.306306306331, 1511.11711711713, 1397.927927927929, 1284.738738738757, 1171.549549549556, 1058.3603603603842, 945.1711711711832, 831.9819819819822], [271.2342342342454, 234.20720720721874, 197.1801801801921, 160.15315315316548, 123.12612612613884, 86.09909909911221, 49.072072072085575, 12.045045045058941], [157.39639639639609, 152.50450450450444, 147.6126126126128, 142.72072072072115, 137.8288288288295, 132.93693693693604, 128.0450450450444, 123.15315315315274], [183.53153153153107, 158.58558558558434, 133.6396396396376, 108.69369369369088, 83.74774774774414, 58.80180180179741, 33.85585585585795, 8.909909909911221]]


39035
CUYAHOGA,OH
Heroin
Oxycodone
Hydrocodone
Fentanyl
Buprenorphine
[[525.0180180179887, 1023.4774774773978, 1521.936936936807, 2020.39639

In [32]:
for fips, num in fips_dict:
    print(fips)
    print(table.loc[table.FIPS_Combined==fips,'COUNTY'].values[0] + ',' + table.loc[table.FIPS_Combined==fips,'State'].values[0])
    print(linear_regression_ridge(get_data_from_FIPS_combined(start_year=2010, end_year=2015, fips=fips)))
    print('\n')

42101
PHILADELPHIA,PA
Heroin
Oxycodone
Codeine
Buprenorphine
Hydrocodone
[[3484.2857142857392, 3615.771428571432, 3747.2571428571828, 3878.7428571428754, 4010.228571428568, 4141.714285714319, 4273.200000000012, 4404.685714285704], [1640.4761904761835, 1520.8190476190648, 1401.161904761917, 1281.504761904769, 1161.8476190476213, 1042.1904761904734, 922.5333333333547, 802.8761904762068], [276.5238095238019, 237.38095238094684, 198.23809523809177, 159.0952380952367, 119.95238095238165, 80.80952380952658, 41.66666666667152, 2.5238095238019014], [158.0952380952367, 152.9238095238088, 147.7523809523791, 142.5809523809512, 137.4095238095233, 132.2380952380936, 127.0666666666657, 121.8952380952378], [187.0952380952367, 160.72380952380627, 134.3523809523831, 107.98095238095266, 81.60952380952222, 55.23809523809905, 28.866666666668607, 2.4952380952381645]]


39035
CUYAHOGA,OH
Heroin
Oxycodone
Hydrocodone
Fentanyl
Buprenorphine
[[453.8095238097012, 980.7523809524719, 1507.6952380954754, 2034.6380